In [ ]:
import yfinance as yf
import numpy as np
import casadi as ca


In [ ]:
TICKERS = ["BTC-USD", "ETH-USD", "XRP-USD", "SOL-USD", "LTC-USD",
           "BNB-USD", "TRX-USD", "ADA-USD", "LINK-USD"]
START = "2020-01-01"
END = "2025-01-01"

loss_threshold = -0.001
tolerance = 1e-6
q = 0  
w_start = None


In [ ]:
data = yf.download(tickers=TICKERS, start=START, end=END)["Close"]
returns = data.pct_change().dropna()

In [ ]:
# -----------------------------
# 1. Portfolio statistics
# -----------------------------
def calculate_mu(data): 
  mu = [] 
  for col in data.columns: 
    mu.append(data[col].mean()) 
  return np.array(mu)

def get_cov(data):
    return np.cov(data.T)

mu_est = calculate_mu(returns)
sigma_est = get_cov(returns)

n_assets = len(mu_est)

# -----------------------------
# 3. Subproblem solver
# -----------------------------
def solve_subproblem(q: float, mu: np.array, sigma: np.array,
                     loss: float, long_only: bool = True):
    n = len(mu)
    w = ca.MX.sym("w", n)

    mu_p = ca.mtimes(w.T, mu)
    var_p = ca.mtimes([w.T, sigma, w])

    f = var_p - q * (mu_p - loss)**2

    g = []
    lb_g, ub_g = [], []

    g.append(ca.mtimes(np.ones((1, n)), w))
    lb_g.append(1)
    ub_g.append(1)

    g = ca.vertcat(*g)
    lb_g = np.array(lb_g)
    ub_g = np.array(ub_g)

    lb_w = np.zeros(n) if long_only else -np.inf * np.ones(n)
    ub_w = np.ones(n)

    nlp = {"x": w, "f": f, "g": g}
    solver = ca.nlpsol("solver", "ipopt", nlp,
                       {"print_time": 0, "ipopt.print_level": 0})
    sol = solver(lbx=lb_w, ubx=ub_w, lbg=lb_g, ubg=ub_g)
    w_opt = np.array(sol["x"]).flatten()
    return w_opt

# -----------------------------
# 4. Dinkelbach loop
# -----------------------------
def dinkelbach_loop(q, mu_est, sigma_est, loss_threshold, max_iter=50):
    for i in range(max_iter):
        w_candidate = solve_subproblem(q, mu_est, sigma_est, loss_threshold)
        mu_p = w_candidate @ mu_est
        var_p = w_candidate.T @ sigma_est @ w_candidate

        q_new = var_p / (mu_p - loss_threshold)**2

        if abs(q_new - q) < tolerance:
            print(f"Converged after {i+1} iterations")
            return mu_p, var_p, w_candidate, q_new

        q = q_new

    print("Max iterations reached, returning last solution")
    return mu_p, var_p, w_candidate, q

In [ ]:
mu_p, var_p, w_star, q = dinkelbach_loop(q, mu_est, sigma_est, loss_threshold)

prob_lower_bound = (mu_p - loss_threshold)**2 / ((mu_p - loss_threshold)**2 + var_p)

print("\nOptimal portfolio weights:")
for i, wi in enumerate(w_star):
    print(f"{TICKERS[i]}: {wi:.4f}")

print(f"\nPortfolio mean return: {mu_p:.6f}")
print(f"Portfolio variance: {var_p:.8f}")
print(f"Guaranteed probability of R >= {loss_threshold}: {prob_lower_bound:.4f}")

portfolio_returns = returns @ w_star
empirical_prob = np.mean(portfolio_returns >= loss_threshold)
print(f"Empirical probability (historical data): {empirical_prob:.4f}")